In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests

数据爬取

In [ ]:
# 创建一个空的DataFrame
df = pd.DataFrame(columns=['Chapter / Instrument No.', 'Chapter / Instrument Title', 'Bilingual_PDFurl', 'Bilingual_RTFurl', 'English_PDFurl', 'English_RTFurl','Chinese_PDFurl', 'Chinese_RTFurl','filepath'])
basic_ref = "https://www.elegislation.gov.hk/"
# 创建一个Selenium WebDriver对象
driver = webdriver.Chrome()  # 根据你的浏览器类型和位置设置

for page_num in range(1,156):

    url = "https://www.elegislation.gov.hk/index/chapternumber?TYPE=1&TYPE=2&TYPE=3&LANGUAGE=E&p0="+str(page_num)

    # 打开网页

    driver.get(url)

    # 设置等待时间，等待页面加载完成
    wait = WebDriverWait(driver, 5)  # 等待时间可以根据需要进行调整
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))

    # 获取页面源码
    page_source = driver.page_source

    # 使用Beautiful Soup解析页面
    soup = BeautifulSoup(page_source, 'html.parser')

    # 进行页面解析和提取操作

    # 找到<tbody>标签
    tbody = soup.find('tbody')
    # 提取<tbody>标签内的内容
    if tbody:
        rows = tbody.find_all('tr')
        for row in rows:
            # 在每一行中查找单元格
            cells = row.find_all('td')
            for i, cell in enumerate(cells):
                # 输出单元格的文本内容
                content = cell.get_text()
                # 根据单元格的位置赋值给变量
                if i == 0:
                    ch_num = content
                elif i == 1:
                    title = content
                elif i == 2:
                    div = cell.find_next('div', class_='download-icons')
                    # 根据<div>标签的存在与否赋值给href
                    if div:
                        href = div.find_all('a')
                        if len(href) >= 2:
                            href_pdf = basic_ref+href[0].get('href')
                            href_rtf = basic_ref+href[1].get('href')
                    else:
                        href_pdf = "-"
                        href_rtf = "-"
                elif i == 3:
                        # 查找单元格内的div元素
                        div = cell.find('div')
                        
                        # 检查div元素是否存在以及其class
                        if div and 'no-download' in div.get('class', []):
                            eng_href_pdf = "-"
                            eng_href_rtf = "-"
                        else:
                            href = div.find_all('a')
                            if len(href) >= 2:
                                eng_href_pdf = basic_ref+href[0].get('href')
                                eng_href_rtf = basic_ref+href[1].get('href')
                elif i == 4:
                        # 查找单元格内的div元素
                        div = cell.find('div')
                        # 检查div元素是否存在以及其class
                        if div and 'no-download' in div.get('class', []):
                            ch_href_pdf = "-"
                            ch_href_rtf = "-"
                        else:
                            href = div.find_all('a')
                            if len(href) >= 2:
                                ch_href_pdf = basic_ref+href[0].get('href')
                                ch_href_rtf = basic_ref+href[1].get('href')
            # 添加一行数据
            
            data_to_add = {'Chapter / Instrument No.': ch_num, 
                        'Chapter / Instrument Title': title, 
                        'Bilingual_PDFurl':href_pdf, 'Bilingual_RTFurl':href_rtf, 
                        'English_PDFurl':eng_href_pdf, 'English_RTFurl':eng_href_rtf,
                        'Chinese_PDFurl':ch_href_pdf, 'Chinese_RTFurl':ch_href_pdf,
                        'filepath':'D:\\download\\'+ ch_num +"-"+ title}
            df = df.append(data_to_add, ignore_index=True)     
    else:
        print("未找到<tbody>标签")

# 关闭浏览器
driver.quit()
# 打印DataFrame
# print(df)
# 保存DataFrame为CSV文件并输出到桌面
desktop_path = "C:/Users/ybq/Desktop/"  # 请替换为你的桌面路径
csv_filename = "dataframe.csv"

df.to_csv(f"{desktop_path}/{csv_filename}", index=False)

print(f"DataFrame已保存为CSV文件: {desktop_path}/{csv_filename}")

文件下载

In [ ]:
# 创建下载PDF文件的函数，接受PDF URL、下载目录和文件名列的名称作为参数
def download_pdf(pdf_url, download_dir, filename_column):
    # 发送HTTP请求下载PDF文件
    response = requests.get(pdf_url)
    if response.status_code == 200:
        # 从DataFrame中获取文件名
        pdf_filename = df.at[index, filename_column]

        # 创建目标文件路径
        pdf_path = os.path.join(download_dir, pdf_filename)

        filename = get_filename(response)
        file_path = pdf_path+filename+'.pdf'
        # 保存文件
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'{file_path} 下载完成')
    else:
        print(f'{pdf_url} 下载失败，HTTP 状态码：{response.status_code}')

def get_filename(response):
    if response.status_code == 200:
        content_disposition = response.headers.get("Content-Disposition")
        if content_disposition:
            filename_match = re.search(r'filename\s*=\s*"([^;"]+)"', content_disposition)
            if filename_match:
                filename = filename_match.group(1).strip()
                return filename
            else:
                print("无法找到文件名")
        else:
            print("无法获取 Content-Disposition 字段")

for index, row in df.iterrows():
    dir = row['filepath']
    if not os.path.exists(dir):
        os.makedirs(dir)
    ch_num = row['Chapter / Instrument No.']
    title = row['Chapter / Instrument Title']

    download_pdf(row['Bilingual_PDFurl'],dir,ch_num +"-"+ title + "-Bilingual.pdf")
    download_pdf(row['Bilingual_RTFurl'],dir,ch_num +"-"+ title + "-Bilingual.rtf")
    if row['English_PDFurl'] != '-':
        download_pdf(row['English_PDFurl'],dir,ch_num +"-"+ title + "-English.pdf")
        download_pdf(row['English_RTFurl'],dir,ch_num +"-"+ title + "-English.rtf")
        download_pdf(row['Chinese_PDFurl'],dir,ch_num +"-"+ title + "-Chinese.pdf")
        download_pdf(row['Chinese_RTFurl'],dir,ch_num +"-"+ title + "-Chinese.rtf")
    